In [36]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

from datetime import datetime
import pandas as pd

os.uname()

posix.uname_result(sysname='Linux', nodename='Acer-JC', release='5.15.133.1-microsoft-standard-WSL2', version='#1 SMP Thu Oct 5 21:02:42 UTC 2023', machine='x86_64')

Now spin up the Selenium-GoogleChrome Docker container. 

`docker start <container id or name>`

In [3]:
def parse_creation_date(datetime_str: str) -> datetime:
    # 6/28/24 11:34 AM => 6/28/2024 11:34 AM
    date_part, time_part, meridiem = datetime_str.split(' ')
    month, day, year = date_part.split('/')
    full_year = f"20{year}"
    new_datetime_str = f"{month}/{day}/{full_year} {time_part} {meridiem}"
    date_format = "%m/%d/%Y %I:%M %p"
    creation_date = datetime.strptime(new_datetime_str, date_format)
    return creation_date

In [39]:
def startScrapeBot_byHTMLclass(username, password, url, last_crawled_datetime, brieftest=False, brieftestLoops=40):
    print('Start scrape bot byHTMLclass....')
    if brieftest:
        print('It is a Brief Test! ')
    # path = "./chrome-win64/chrome.exe"
    # giving the path of chromedriver to selenium webdriver
    # Set up Chrome options
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--disable-infobars")
    # chrome_options.add_argument("--disable-notifications")
    # chrome_options.add_argument("--disable-extensions")
    # chrome_options.add_argument("--start-maximized")
    # chrome_options.add_argument("--disable-infobars")
    
    chrome_options.add_argument("--disable-features=SidePanelPinning")
    
    # chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    # chrome_options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Remote(
        command_executor='http://localhost:4444/wd/hub',
        options=chrome_options # webdriver.ChromeOptions()
    )

    # opening the website  in chrome.
    print('Opening URL....')
    try:
        driver.get(url)
        assert "iExchangeWeb" in driver.title
    except:
        driver.quit()

    print('find the id "login-box"....')
    try:
        loginBox = WebDriverWait(driver, 60).until( \
            EC.presence_of_element_located((By.ID, "login-box")))
        print('found loginbox!')
    except:
        driver.quit()
    
    # find the id or name or class of
    # username by inspecting on username input
    print('find the id "userName" within loginBox....')
    try:
        userNameForm = WebDriverWait(loginBox, 60).until( \
            EC.presence_of_element_located((By.ID, "userName")))
        print('found userNameForm!')
        userNameForm.send_keys(username)
        print('sent userName!')
    except:
        print('error at: have not sent userName')
        driver.quit()
     
    # find the password by inspecting on password input
    print('find the id "password" within loginBox....')
    try:
        passwordForm = WebDriverWait(loginBox, 60).until( \
            EC.presence_of_element_located((By.ID, "password")))
        print('found password!')
        passwordForm.send_keys(password)
        print('sent password!')
    except:
        print('error at: have not sent password')
        driver.quit()
    
    # click on submit
    print('clicking on submit after finding userName and password....')
    try:
        signin_button = WebDriverWait(loginBox, 60).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, "#login-box .btn-primary"))
                        )
        print('found signin_button! ')
    except Exception as e:
        print(f'error at: signin_button = WebDriverWait(driver, 60).until( \n{e}')
        driver.quit()

    try:
        assert signin_button.text == 'Sign In'
        print('Assured signin button has the text "Sign in", prepare to click!')
        signin_button.click()
    except Exception as e:
        print('error at: assert signin_button.text == Sign in')
        print(e)
        driver.quit()

    # New Page: mailbox/inbox
    try:
        assert WebDriverWait(driver, 60).until(EC.url_contains('mailbox/inbox'))
        print('successful login to mailbox!')
    except Exception as e:
        print(f'error at: "assert mailbox/inbox in mailbox_url"')
        print(e)
        driver.quit()

    # Locate the sentmail_button
    try:
        leftside_bar = WebDriverWait(driver, 60).until( \
            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/aside[1]')))
        mailbox_button = WebDriverWait(leftside_bar, 60).until( \
            EC.presence_of_element_located((By.XPATH, './/section/ul/li[1]/a')))
        # find_element(By.XPATH, '/html/body/div[2]/aside[1]/section/ul/li[1]/a')
        print('found mailbox_button, prepare to click! ')
        mailbox_button.click()
    except NoSuchElementException as nsee:
        print('error at: left_sidebar = driver.find_element...')
        print(nsee)
        driver.quit()

    try:
        sentmail_button = WebDriverWait(leftside_bar, 60).until(
                            EC.element_to_be_clickable((By.XPATH, ".//section/ul/li[1]/ul/li[4]/a"))
                        )
        print('found Sent button! ')
    except Exception as e:
        print('error at: sentmail_button = WebDriverWait(driver, 60).unti')
        print(e)
        driver.quit() 

    try:
        # assert WebDriverWait(sentmail_button, 60).until(
        #                     EC.presence_of_element_located((By.XPATH, ".//span"))
        #                 ).text == "Sent"
        assert WebDriverWait(sentmail_button, 60).until(
            EC.text_to_be_present_in_element((By.TAG_NAME, "span"), "Sent")
        )
        print('Assured that the text "Sent" is present in the button!, prepared to click...')
        sentmail_button.click()
    except Exception as e:
        print('error at: assert sentmail_button.find_element(By.XPATH, ')
        print(e)
        driver.quit() 
    
    try:
        assert WebDriverWait(driver, 60).until(EC.url_contains('mailbox/sent'))
        print('Navigated to Sent mail page!')
        assert WebDriverWait(driver, 60).until(
            EC.text_to_be_present_in_element((By.XPATH, "/html/body/div[2]/aside[2]/ol/li[2]"), "Sent")
        )
        print('Mailbox/Sent is in title! ')
        sentmail_url = driver.current_url
        print(sentmail_url)
    except Exception as e:
        print('error at: assert WebDriverWait(driver, 60).until(EC.url_contains(mailbox/sent)) or is_title_sent')
        print(e)
        driver.quit()
    
    # New page, driver current page redirected! 
    # Inside sent mails, find the rows where Subject="Accepted -Ship Notice....."
    
    # Locate the <tbody> tag where the table is at
    try:
        tbody = WebDriverWait(driver, 60).until(
            EC.visibility_of_element_located((By.TAG_NAME, 'tbody'))
        )
        print('found tbody tag! ')
    except:
        driver.quit()
    # Get all the rows that all contain a <tr> tag
    try:     
        rows = WebDriverWait(tbody, 60).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
        )
        print(f'got all rows! row count={len(rows)}')
    except:
        print('error at: Get all the rows that all contain a <tr> tag')
        driver.quit()

    # Out of all the rows, find the "un-crawled" rows
    ship_notice_idx=[]    
    for i, row in enumerate(rows):
        print(f'getting uncrawled row {i}....')
        if brieftest and i==brieftestLoops: break

        try:
            creation_date_str = WebDriverWait(row, 60).until( \
                EC.presence_of_element_located((By.XPATH, "./td[11]"))).text
            creation_date = parse_creation_date(creation_date_str)
            if last_crawled_datetime >= creation_date:
                print('last_crawled_datetime >= creation_date. done. ')
                break
            print(f'found and parsed creation_date_str: {creation_date}')
            subject_str = WebDriverWait(row, 60).until( \
                EC.presence_of_element_located((By.XPATH, "./td[10]"))).text
            # print(f'found subject_str: {subject_str}')
        except Exception as e:
            print('error at: subject_str = row.find_element(By.XPATH,"./td[10]").text')
            print(e)
            driver.quit()
        
        try:
            if subject_str.startswith('Accepted -Ship Notice'):
                print(f'{subject_str}')
                ship_notice_idx.append(i)
            else: continue
            
        except Exception as e:
            print('error at: subject_str.startswith("Accepted -Ship Notice")')
            print(e)
            driver.quit()
    
    # In each un-crawled ship notice, click on its "view" folder-like button and get data from classes = 'caption', 'data'. 
    redflag=0
    df_shipNotice = pd.DataFrame()
    print(f'crawled ship notices at rows = {ship_notice_idx}....')
    tmp=0
    for idx in reversed(ship_notice_idx): # reversed, start processing from earliest non-crawled date.
        if tmp != 0:
            break
        tmp+=1
        # try:
        #     subject_str = WebDriverWait(row, 60).until( 
        #             EC.presence_of_element_located((By.XPATH, "./td[10]"))).text
        #     print(f'subject_str = {subject_str} ./td[10]')
        # except Exception as e:
        #     print(f'error at: subject_str = {subject_str} ./td[10]')
        #     print(e)
        #     driver.quit()
        assert WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.TAG_NAME, "tbody")))
        print('waited for tbody to appear in the mailbox/sent page')
        


        # To avoid stale element exception, find all rows everytime. 
        try:
            tbody = WebDriverWait(driver, 60).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/aside[2]/section/div/table/tbody'))
            )
            rows = WebDriverWait(tbody, 60).until(
                EC.presence_of_all_elements_located((By.XPATH, './tr')) # only "direct child" under tbody
            )
        except Exception as e:
            print('error at: avoid stale element exception, find all rows everytime. ')
            print(e)
            driver.quit()

        print(f'rows count = {len(rows)}')
        row = rows[idx]

        # click on View Button and direct to EDI item page. 
        try:
            # !!!!! view button can't be clicked, and rows index not working as expected. 


            # view_button = WebDriverWait(row, 60).until(
            #     EC.element_to_be_clickable((By.CSS_SELECTOR, '[title="View"]'))
            # )

            # # If access through absolute XPATH to the same view button, it clicks and works
            # view_button = WebDriverWait(row, 60).until(
            #     EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/aside[2]/section/div/table/tbody/tr[95]/td[14]/button[1]'))
            #     # EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/aside[2]/section/div/table/tbody/tr[95]/td[14]/button[1]'))
            # )

            # # If access through relative XPATH?
            # view_button = WebDriverWait(row, 60).until(
            #     EC.element_to_be_clickable((By.XPATH, './td[14]/button[1]'))
            # )

            view_button = WebDriverWait(row, 60).until(
                EC.element_to_be_clickable((By.XPATH, '//button[@title="View"]'))
            )
            
            print('view_button is clickable')
            view_button.click()
            # ActionChains(driver).move_to_element(view_button).click()
        except ElementClickInterceptedException:
            # Click using JavaScript as a fallback
            print('Click using JavaScript as a fallback')
            driver.execute_script("arguments[0].click();", view_button)
        except Exception as e:
            print('error at: view_button click!')
            print(e)
            driver.quit()

        try:
            assert WebDriverWait(driver, 60).until(
                EC.url_contains('mailbox/item')
            )
            print('url contains mailbox/item (inside view)')
            assert WebDriverWait(driver, 60).until( \
                EC.visibility_of_all_elements_located((By.XPATH, "/html/body/div[2]/aside[2]/section"))
            )
            print(f'visibility_of_all_elements_located((By.XPATH, "/html/body/div[2]/aside[2]/section")')
            assert WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.ID, 'contentFrame'))
            ), "contentFrame is NOT seen!!!"
            print('contentFrame is Seen! ')
            
        except Exception as e:
            print('error at: assert WebDriverWait(driver, 60).until(EC.url_contains(mailbox/item))')
            print(e)
            driver.quit()



        # Get the desired data
        try:
            # Need to Switch to iFrame first, because all the data (#document) is in under an iframe! 
            # Find the iframe element by id
            try:
                iframe_locator = WebDriverWait(driver, 60).until(
                    EC.presence_of_element_located((By.ID, 'contentFrame'))
                )
                print('iframe_locator succeed')
            except Exception as e:
                print(f'error at: iframe_locator. \nerror={e}')
                driver.quit()
            # Switch to the iframe context
            try:
                WebDriverWait(driver, 60).until(
                    # If the frame is available it switches the given driver to the specified frame.
                    EC.frame_to_be_available_and_switch_to_it(iframe_locator)
                )
                print('iframe_locator successfully switched!')
            except Exception as e:
                print(f'error at: frame_to_be_available_and_switch_to_it(iframe_locator). \nerror={e}')
                driver.quit()
            # driver now represents iframe
            try:
                ship_notice_body = WebDriverWait(driver, 60).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body'))
                )
                print('driver now represents iframe, iframe body located!')
            except Exception as e:
                print(f'error at: driver now represents iframe, ship_notice_body. \nerror={e}')
                driver.quit()
            
            # get all tables
            try:
                tables = WebDriverWait(ship_notice_body, 60).until(
                    EC.presence_of_all_elements_located((By.XPATH, './table'))
                )
                print('Got all tablesss!')
            except Exception as e:
                print(f'error at:  get all tables. \nerror={e}')
                driver.quit()


            # for j, table in enumerate(tables):
            #     if j==0:
            #         ship_notice_title = WebDriverWait(table, 60).until(
            #             EC.presence_of_element_located((By.XPATH, './tbody/tr[1]/td/h1'))
            #         ).text
            #         print(f'Title: {ship_notice_title}')


            # start retrieving desired data in current row/ship_notice
            sharedAttr_dict = dict()
            print(f'tables count = {len(tables)} !!')
            for j, table in enumerate(tables):
                print(f'get Table {j}')
                if j==0:
                    ship_notice_title = WebDriverWait(table, 60).until(
                        EC.presence_of_element_located((By.XPATH, './tbody/tr[1]/td/h1'))
                    ).text
                    print(f'Title: {ship_notice_title}')
                
                else:
                    upperleftmost_element = WebDriverWait(table, 60).until(
                        EC.presence_of_element_located((By.XPATH, "./tbody/tr/td[1]/table/tbody/tr[1]"))
                    )
                    upperleftmost_caption_element = WebDriverWait(upperleftmost_element, 60).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "caption"))
                    )
                    upperlefmost_caption = upperleftmost_caption_element.text.strip()
                    if upperlefmost_caption == "Ship To":
                        data_element = WebDriverWait(upperleftmost_caption_element, 60).until(
                            EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                        )
                        sharedAttr_dict['ship_to'] = data_element.text
                        assert 'ship_to' in sharedAttr_dict.keys(), "'ship_to' key not found in sharedAttr_dict"
                    elif upperlefmost_caption == "Ship Notice #":
                        # get all elements with "caption" classes in the table.tbody element (table.tbody)
                        all_caption_elements = WebDriverWait(table, 60).until(
                            EC.presence_of_all_elements_located((By.CLASS_NAME, "caption"))
                        )
                        target_attr_count=0
                        for caption_element in all_caption_elements:
                            print(f'Ship Notice # captions  =  {caption_element.text}')
                            if caption_element.text.strip() == 'Ship Notice #':
                                # find sibling (data)
                                data_element = WebDriverWait(caption_element, 60).until(
                                    EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                                )
                                sharedAttr_dict['ship_notice_num'] = data_element.text
                                target_attr_count+=1
                            elif caption_element.text.strip() == 'Create Date/Time':
                                # find sibling (data)
                                data_element = WebDriverWait(caption_element, 60).until(
                                    EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                                )
                                sharedAttr_dict['create_datetime'] = data_element.text
                                target_attr_count+=1
                            elif target_attr_count==2: break
                        assert 'ship_notice_num' in sharedAttr_dict.keys(), "'ship_notice_num' key not found in sharedAttr_dict"
                        assert 'create_datetime' in sharedAttr_dict.keys(), "'create_datetime' key not found in sharedAttr_dict"
                    elif upperlefmost_caption == "Container Type": 
                        continue
                    elif "Order #" in upperlefmost_caption or "PO #" in upperlefmost_caption: # items
                        itemAttr_dict = {}
                        all_caption_elements = WebDriverWait(table, 60).until(
                            EC.presence_of_all_elements_located((By.CLASS_NAME, "caption"))
                        )
                        for caption_element in all_caption_elements:
                            print(f'Order #, PO # captions  =  {caption_element.text}')
                            caption = caption_element.text
                            if "Order #" in caption or "PO #" in caption:
                                # find sibling (data)
                                
                                data_element = WebDriverWait(caption_element, 60).until(
                                    EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                                )
                                if 'Order #' in caption:
                                    itemAttr_dict['order_type'] = 'Buyer'
                                else:
                                    itemAttr_dict['order_type'] = 'Purchase'
                                print(f'yay Im in Order # and PO #!!!\ndata={data_element.text}' )
                                itemAttr_dict['order_num'] = data_element.text
                                target_attr_count+=1
                            elif 'Buyer Part #' in caption: 
                                data_element = WebDriverWait(caption_element, 60).until(
                                    EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                                )
                                itemAttr_dict['buyer_part_num'] = data_element.text
                                target_attr_count+=1
                            elif 'Ship Quantity' in caption: 
                                data_element = WebDriverWait(caption_element, 60).until(
                                    EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                                )
                                itemAttr_dict['ship_quantity'] = data_element.text
                                target_attr_count+=1
                            elif target_attr_count==3:
                                break
                        assert 'order_num' in itemAttr_dict.keys(), "'order_num' key not found in itemAttr_dict"
                        assert 'order_type' in itemAttr_dict.keys(), "'order_type' key not found in itemAttr_dict"
                        assert 'buyer_part_num' in itemAttr_dict.keys(), "'buyer_part_num' key not found in itemAttr_dict"
                        assert 'ship_quantity' in itemAttr_dict.keys(), "'ship_quantity' key not found in itemAttr_dict"
                        
                        # combine a attributes for a single item at a column level
                        df_row = pd.DataFrame.from_dict([{**sharedAttr_dict, **itemAttr_dict}])
                        # concat items to a allItems ship notice list at a row level
                        df_shipNotice = pd.concat([df_shipNotice, df_row], ignore_index=True)
                    else:
                        print(f'dont need to use table {upperlefmost_caption} right now...')
                    

                # ####################use attribute name to decide rather than index j########################
                # elif j==1: # table1, get "Ship To"
                #     caption_elements = WebDriverWait(table, 60).until(
                #         EC.presence_of_all_elements_located((By.CLASS_NAME, "caption"))
                #     )

                #     for caption_element in caption_elements:
                #         print(f'j==1 captions  =  {caption_element.text}')
                #         if caption_element.text.strip() == 'Ship To':
                #             data_element = WebDriverWait(caption_element, 60).until(
                #                 EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                #             )
                #             sharedAttr_dict['ship_to'] = data_element.text
                #             # break

                #     assert 'ship_to' in sharedAttr_dict.keys(), "'ship_to' key not found in sharedAttr_dict"
                
                # elif j==2: # table2, get "Ship Notice #", "Create Date/Time"
                #     caption_elements = WebDriverWait(table, 60).until(
                #         EC.presence_of_all_elements_located((By.CLASS_NAME, "caption"))
                #     )
                #     target_attr_count=0
                #     for caption_element in caption_elements:
                #         print(f'j==2 captions  =  {caption_element.text}')
                #         if caption_element.text.strip() == 'Ship Notice #':
                #             # find sibling (data)
                #             data_element = WebDriverWait(caption_element, 60).until(
                #                 EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                #             )
                #             sharedAttr_dict['ship_notice_num'] = data_element.text
                #             target_attr_count+=1
                #         elif caption_element.text.strip() == 'Create Date/Time':
                #             # find sibling (data)
                #             data_element = WebDriverWait(caption_element, 60).until(
                #                 EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                #             )
                #             sharedAttr_dict['create_datetime'] = data_element.text
                #             target_attr_count+=1
                #         elif target_attr_count==2: break                  

                #     assert 'ship_notice_num' in sharedAttr_dict.keys(), "'ship_notice_num' key not found in sharedAttr_dict"
                #     assert 'create_datetime' in sharedAttr_dict.keys(), "'create_datetime' key not found in sharedAttr_dict"
                
                # elif j==3: # skip. (container type, master label #, returnable container #)
                #     continue
                # else: # table4~N, items
                #     itemAttr_dict = {}

                #     caption_elements = WebDriverWait(table, 60).until(
                #         EC.presence_of_all_elements_located((By.CLASS_NAME, "caption"))
                #     )
                    
                #     target_attr_count=0
                #     for caption_element in caption_elements:
                #         print(f'target_attr_count={target_attr_count}')
                #         print(f'j>3 captions  =  {caption_element.text}')
                #         caption = caption_element.text
                #         if "Order #" in caption or "PO #" in caption:
                #             # find sibling (data)
                            
                #             data_element = WebDriverWait(caption_element, 60).until(
                #                 EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                #             )
                #             if 'Order #' in caption:
                #                 itemAttr_dict['order_type'] = 'Buyer'
                #             else:
                #                 itemAttr_dict['order_type'] = 'Purchase'
                #             print(f'yay Im in Order # and PO #!!!\ndata={data_element.text}' )
                #             itemAttr_dict['order_num'] = data_element.text
                #             target_attr_count+=1
                #         elif 'Buyer Part #' in caption: 
                #             data_element = WebDriverWait(caption_element, 60).until(
                #                 EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                #             )
                #             itemAttr_dict['buyer_part_num'] = data_element.text
                #             target_attr_count+=1
                #         elif 'Ship Quantity' in caption: 
                #             data_element = WebDriverWait(caption_element, 60).until(
                #                 EC.presence_of_element_located((By.XPATH, "following-sibling::td[@class='data']"))
                #             )
                #             itemAttr_dict['ship_quantity'] = data_element.text
                #             target_attr_count+=1
                #         elif target_attr_count==3:
                #             break
                    
                    
                #     assert 'order_num' in itemAttr_dict.keys(), "'order_num' key not found in itemAttr_dict"
                #     assert 'order_type' in itemAttr_dict.keys(), "'order_type' key not found in itemAttr_dict"
                #     assert 'buyer_part_num' in itemAttr_dict.keys(), "'buyer_part_num' key not found in itemAttr_dict"
                #     assert 'ship_quantity' in itemAttr_dict.keys(), "'ship_quantity' key not found in itemAttr_dict"

                #     # combine a attributes for a single item at a column level
                #     df_row = pd.DataFrame.from_dict([{**sharedAttr_dict, **itemAttr_dict}])
                #     # concat items to a allItems ship notice list at a row level
                #     df_shipNotice = pd.concat([df_shipNotice, df_row], ignore_index=True)



            # single data row looks like {
            #     "ship_to": "Location A",
            #     "ship_notice_num": "12345",
            #     "create_datetime": "2024-07-10 05:32:00",
            #     "order_num": "54321",
            #     "order_type": "Purchase or Buyer"
            #     "buyer_part_num": "67890",
            #     "ship_quantity": 100
            # }

        except Exception as e:
            print('Exception occured, saving to temporary csv')
            redflag=1
            df_shipNotice.to_csv('./shipnotices/ship-notice-tmp.csv')
            print(df_shipNotice.head())
            print('error at: Get the desired data')
            print(e)
            driver.quit()
            break

        # switch back to main mailbox page
        try:    
            # if needed to switchback to original parent frame from iframe
            # driver.switch_to.default_content()

            # Navigate back to the previous page
            driver.get(sentmail_url)

            # Wait until the URL changes back to the previous one (optional)
            WebDriverWait(driver, 60).until(EC.url_to_be(sentmail_url))
            assert WebDriverWait(driver, 60).until(EC.url_contains('mailbox/sent'))
            print('Redirected to Sent mail page!')
            assert WebDriverWait(driver, 60).until(
                EC.text_to_be_present_in_element((By.XPATH, "/html/body/div[2]/aside[2]/ol/li[2]"), "Sent")
            )
            print('Mailbox/Sent is in title! ')

        except Exception as e:
            print('error at: back to mailbox page')
            print(e)
            driver.quit()

    if not redflag:
        df_shipNotice.to_csv('./shipnotices/ship-notice.csv', index_label='id')
    driver.quit()



In [40]:
# Driver Code
# Enter below your login credentials
username = "***REMOVED***"
password = "***REMOVED***"

# URL of the login page of site
# which you want to automate login.
url = "https://www.iexchangeweb.com/ieweb/general/login"

# Get the "new data": last datetime of crawled data ~ newest
date_format = "%m/%d/%Y %I:%M %p"
last_crawled_datetime = datetime.strptime("7/1/2024 2:28 PM", date_format)

# Call the function
# startScrapeBot_byRawX12(username, password, url, last_crawled_datetime, brieftest=True)
startScrapeBot_byHTMLclass(username, password, url, last_crawled_datetime, brieftest=True)

last_crawled_datetime = datetime.today()
print(f'last_crawled_datetime={last_crawled_datetime}')
# goto http://localhost:7900/?autoconnect=1&resize=scale&password=secret

Start scrape bot byHTMLclass....
It is a Brief Test! 
Opening URL....
find the id "login-box"....
found loginbox!
find the id "userName" within loginBox....
found userNameForm!
sent userName!
find the id "password" within loginBox....
found password!
sent password!
clicking on submit after finding userName and password....
found signin_button! 
Assured signin button has the text "Sign in", prepare to click!
successful login to mailbox!
found mailbox_button, prepare to click! 
found Sent button! 
Assured that the text "Sent" is present in the button!, prepared to click...
Navigated to Sent mail page!
Mailbox/Sent is in title! 
https://www.iexchangeweb.com/ieweb/exchange/mailbox/sent
found tbody tag! 
got all rows! row count=100
getting uncrawled row 0....
found and parsed creation_date_str: 2024-07-16 03:22:00
getting uncrawled row 1....
found and parsed creation_date_str: 2024-07-16 03:22:00
getting uncrawled row 2....
found and parsed creation_date_str: 2024-07-16 03:19:00
getting unc

InvalidSessionIdException: Message: Unable to find session with ID: 25473d2442e01a05058a1499dca9076a
Build info: version: '4.22.0', revision: 'c5f3146703'
System info: os.name: 'Linux', os.arch: 'amd64', os.version: '5.15.133.1-microsoft-standard-WSL2', java.version: '17.0.11'
Driver info: driver.version: unknown
Stacktrace:
org.openqa.selenium.NoSuchSessionException: Unable to find session with ID: 25473d2442e01a05058a1499dca9076a
Build info: version: '4.22.0', revision: 'c5f3146703'
System info: os.name: 'Linux', os.arch: 'amd64', os.version: '5.15.133.1-microsoft-standard-WSL2', java.version: '17.0.11'
Driver info: driver.version: unknown
	at org.openqa.selenium.grid.sessionmap.local.LocalSessionMap.get(LocalSessionMap.java:132)
	at org.openqa.selenium.grid.sessionmap.SessionMap.getUri(SessionMap.java:84)
	at org.openqa.selenium.grid.router.HandleSession.lambda$loadSessionId$4(HandleSession.java:223)
	at io.opentelemetry.context.Context.lambda$wrap$2(Context.java:224)
	at org.openqa